<a href="https://colab.research.google.com/github/KoreTane/interesting/blob/main/importxtr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import win32com.client as win32
import pdfplumber
from PyPDF2 import PdfMerger

# Função para ler o conteúdo de um arquivo PDF usando pdfplumber
def ler_pdf(caminho_pdf):
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            conteudo = ''
            for page in pdf.pages:
                conteudo += page.extract_text() + '\n'
        return conteudo
    except Exception as e:
        print(f"Erro ao ler o arquivo {caminho_pdf}: {e}")
        return ""

# Função para extrair o número da NFE
def extrair_nfe(conteudo):
    match = re.search(r'\b(\d{3})\.(\d{3})\.(\d{3})\b', conteudo)
    if match:
        nfe_numero = ''.join(match.groups()).lstrip('0')
        return nfe_numero
    return None

# Função para extrair o número do BOLETO
def extrair_boleto(conteudo):
    match = re.search(r'\b(\d{5})-(\d)', conteudo)
    if match:
        boleto_numero = match.group(1).lstrip('0')
        return boleto_numero
    return None

# Função para mesclar PDFs
def mesclar_pdfs(caminho_nfe, caminho_boleto, caminho_saida):
    merger = PdfMerger()
    merger.append(caminho_nfe)
    merger.append(caminho_boleto)
    merger.write(caminho_saida)
    merger.close()

# Função para buscar NFEs e Boletos no e-mail e processar os anexos
def buscar_nfes_email():
    outlook = win32.Dispatch('outlook.application')
    inbox = outlook.GetNamespace("MAPI").GetDefaultFolder(6)  # 6 é o índice da pasta "Caixa de Entrada"

    for mensagem in inbox.Items:
        try:
            # Verifica se o item é um e-mail (MailItem) com anexos
            if mensagem.Class == 43:  # 43 é o código para MailItem
                nfe_encontrada = False
                boleto_encontrado = False

                nfe_caminho = None
                boleto_caminho = None

                for anexo in mensagem.Attachments:
                    if anexo.FileName.endswith(".pdf"):
                        # Salva temporariamente o anexo para análise
                        caminho_temp = os.path.join(r"C:\Users\Public\PDF\TEMP", anexo.FileName)
                        os.makedirs(os.path.dirname(caminho_temp), exist_ok=True)
                        anexo.SaveAsFile(caminho_temp)

                        conteudo_pdf = ler_pdf(caminho_temp)
                        nfe_numero = extrair_nfe(conteudo_pdf)
                        boleto_numero = extrair_boleto(conteudo_pdf)

                        # Verifica se ambos foram encontrados
                        if nfe_numero:
                            nfe_encontrada = True
                            nfe_caminho = caminho_temp  # Armazena o caminho da NFE

                            # Nome do remetente normalizado para uso em caminhos de diretório
                            remetente = mensagem.SenderName.replace(" ", "_")
                            pasta_destino = os.path.join(r"C:\Users\Public\PDF", remetente)
                            os.makedirs(pasta_destino, exist_ok=True)

                            # Renomeia e move a NFE encontrada
                            novo_caminho_nfe = os.path.join(pasta_destino, f"NFE_{nfe_numero}.pdf")
                            os.rename(caminho_temp, novo_caminho_nfe)  # Move NFE
                            print(f"NFE encontrada e salva: {novo_caminho_nfe}")
                            nfe_caminho = novo_caminho_nfe  # Atualiza o caminho da NFE

                        if boleto_numero:
                            boleto_encontrado = True

                            # Salva novamente como PDF do BOLETO (usando o mesmo caminho temporário)
                            novo_caminho_boleto = os.path.join(pasta_destino, f"BOLETO_{boleto_numero}.pdf")
                            anexo.SaveAsFile(novo_caminho_boleto)  # Salva BOLETO diretamente do anexo
                            print(f"BOLETO encontrado e salvo: {novo_caminho_boleto}")
                            boleto_caminho = novo_caminho_boleto  # Atualiza o caminho do BOLETO

                # Se ambos foram encontrados, mescla os arquivos PDF e remove os originais
                if nfe_encontrada and boleto_encontrado and nfe_caminho and boleto_caminho:
                    caminho_saida_mesclado = os.path.join(pasta_destino, f"Mesclado_{nfe_numero}_{boleto_numero}.pdf")
                    mesclar_pdfs(nfe_caminho, boleto_caminho, caminho_saida_mesclado)
                    print(f"NFE e BOLETO mesclados salvos em: {caminho_saida_mesclado}")

                    # Remove os arquivos individuais após a mesclagem
                    os.remove(nfe_caminho)
                    os.remove(boleto_caminho)

                else:
                    print(f"Nenhuma NFE ou BOLETO encontrados nos anexos do e-mail de: {mensagem.SenderName}")

        except Exception as e:
            print(f"Erro ao processar a mensagem: {e}")

# Cria o diretório principal se não existir
os.makedirs(r"C:\Users\Public\PDF", exist_ok=True)

# Processar os anexos de e-mails e salvar as NFEs e Boletos identificados
buscar_nfes_email()

print("Processo concluído. Confira as NFEs e Boletos em C:\\Users\\Public\\PDF.")